## Idea:
* Use month trend from [Only Trends](https://www.kaggle.com/code/vitalykudelya/only-trends)
* Divide NPX values of a protein into several groups and find the best shift after month trend predicitons for each group
* Sum predictions from the month trend and the corresponding NPX group shift

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pickle
import gc

import optuna
from optuna.samplers import TPESampler
from sklearn.svm import SVR, LinearSVR

import plotly.express as px

import amp_pd_peptide

from scipy.optimize import minimize
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

import lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation


target_col = ['updrs_1','updrs_2','updrs_3','updrs_4']

pd.set_option('display.max_columns', 500) ## 모든 열 출력
pd.set_option('display.max_rows', 500) ## 모든 행 출력

In [2]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)


def smape(y_true, y_pred):

    metric = np.zeros(len(y_true))
    
    numerator = np.abs(y_true - y_pred)
    denominator = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    mask_not_zeros = (y_true != 0) | (y_pred != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

## Generate Train Dataset

In [3]:
train_clinical_all = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
supplemental_clinical_data = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
proteins_features = pd.pivot_table(proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')

train_clinical_all = pd.merge(train_clinical_all,
    proteins_features,
    on='visit_id',
    how='left'
)


In [4]:
# peptides[peptides['UniProt']=='P05060'].pivot('visit_id','Peptide','PeptideAbundance').reset_index().isnull().sum()

In [5]:
# merge_df = pd.merge(proteins.pivot('visit_id','UniProt','NPX').reset_index()[['visit_id','P05060']],
#          peptides[peptides['UniProt']=='P05060'].pivot('visit_id','Peptide','PeptideAbundance').reset_index(),
#          on='visit_id',
#          how='left')


In [6]:
# merge_df['P05060_except_CIIEVL'] = merge_df[["GEAGAPGEEDIQGPTK","GYPGVQAPEDLEWER",\
#                                            'LLRDPADASEAHESSSR','SSQGGSLPSEEK']].sum(axis=1)

# merge_df['P05060_except_GEAGA'] = merge_df[["C(UniMod_4)IIEVLSNALSK","GYPGVQAPEDLEWER",\
#                                            'LLRDPADASEAHESSSR','SSQGGSLPSEEK']].sum(axis=1)

# merge_df['P05060_except_GYPGVO'] = merge_df[["GEAGAPGEEDIQGPTK","C(UniMod_4)IIEVLSNALSK",\
#                                            'LLRDPADASEAHESSSR','SSQGGSLPSEEK']].sum(axis=1)

# merge_df['P05060_except_LLRDP'] = merge_df[["GEAGAPGEEDIQGPTK","GYPGVQAPEDLEWER",\
#                                            'C(UniMod_4)IIEVLSNALSK','SSQGGSLPSEEK']].sum(axis=1)

# merge_df['P05060_except_SSQGG'] = merge_df[["GEAGAPGEEDIQGPTK","GYPGVQAPEDLEWER",\
#                                            'LLRDPADASEAHESSSR','C(UniMod_4)IIEVLSNALSK']].sum(axis=1)



In [7]:
# merge_df.drop(["GEAGAPGEEDIQGPTK","GYPGVQAPEDLEWER",'LLRDPADASEAHESSSR','C(UniMod_4)IIEVLSNALSK','SSQGGSLPSEEK'],axis=1).corr()

In [8]:
# train_clinical_all = pd.merge(train_clinical_all, merge_df, on='visit_id',how='left')

In [9]:
# train_clinical_all.corr()[target_col]

In [10]:
# train_clinical_all

In [11]:
# 중앙값
target_columns_clinical_data = ['updrs_1']
target_columns_clinical_and_supplemental_data = ['updrs_2', 'updrs_3', 'updrs_4']

target_visit_month_medians_clinical_data = train_clinical_all.groupby('visit_month')[target_columns_clinical_data].median()
target_visit_month_medians_clinical_and_supplemental_data = pd.concat((
    train_clinical_all,
    supplemental_clinical_data
), axis=0).groupby('visit_month')[target_columns_clinical_and_supplemental_data].median()

target_visit_month_medians = pd.concat((
    target_visit_month_medians_clinical_data,
    target_visit_month_medians_clinical_and_supplemental_data
), axis=1, ignore_index=False)

target_visit_month_medians = target_visit_month_medians.drop(5)
target_visit_month_medians = target_visit_month_medians.drop(3)
target_visit_month_medians = target_visit_month_medians.drop(9)

target_visit_month_medians = target_visit_month_medians.expanding(min_periods=1).max()
target_visit_month_medians = target_visit_month_medians.reset_index()
target_visit_month_medians

,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,0,4.5,4.0,18.0,0.0
1,6,6.0,6.0,21.0,0.0
2,12,6.0,6.0,21.0,0.0
3,18,6.0,6.0,21.0,0.0
4,24,6.0,6.0,21.0,0.0
5,30,7.0,6.0,22.0,0.0
6,36,7.0,6.0,22.0,0.0
7,42,7.0,7.0,23.0,0.0
8,48,7.0,7.0,23.0,0.0
9,54,7.0,9.0,26.0,0.0


In [12]:
train_clinical_all[proteins_features.columns] = train_clinical_all.groupby('patient_id')[proteins_features.columns].\
                                                                                        fillna(method='ffill')
                                                 


In [13]:
# feature = 'P05060'
train_clinical_all['pred_month'] = train_clinical_all['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = train_clinical_all[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
#     train_shift.rename(columns={f'{feature}': f'{feature}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
#     display(train_shift)
    train_clinical_all = train_clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
# train_clinical_all.rename(columns={f'{feature}': f'{feature}_plus_0'}, inplace=True)
train_clinical_all.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)
train_clinical_all

,visit_id,patient_id,visit_month,updrs_1_plus_0,updrs_2_plus_0,updrs_3_plus_0,updrs_4_plus_0,upd23b_clinical_state_on_medication,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05408,P05452,P05546,P06310,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09486,P09871,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P23083,P23142,P24592,P25311,P27169,P30086,P31997,P32754,P35542,P36222,P36955,P36980,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99683,Q99829,Q99832,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,pred_month_plus_0,pred_month_plus_6,updrs_1_plus_6,updrs_2_plus_6,updrs_3_plus_6,updrs_4_plus_6,pred_month_plus_12,updrs_1_plus_12,updrs_2_plus_12,updrs_3_plus_12,updrs_4_plus_12,pred_month_plus_24,updrs_1_plus_24,updrs_2_plus_24,updrs_3_plus_24,updrs_4_plus_24
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,11254.3,732430.0,39585.8,41526.9,31238.0,4202.71,177775.0,62898.2,333376.0,166850.0,98746.3,6380.35,67567.6,64117.8,1181230.0,688909.0,109541.0,3956470.0,111619.0,347865.0,71835.4,637630.0,2676370.0,14415900.0,2025890.0,1984650.0,1953020.0,3916980.0,13033.3,117641.0,43235500.0,1221530.0,136544.0,6978.98,NaN,11386.00,126829.0,110750.0,9600.69,4950980.0,2121640.0,3521370.0,2224420.0,714486.0,18430.1,1829650.0,39221.5,87890.4,11270600.0,15487900.0,842865.0,24917.0,88723.3,668070.0,568133.0,209127.0,161430.0,162768.0,1613890.0,370958.0,2541830.0,571002.0,396955.0,5584050.0,2625460.0,23735400.0,390239000.0,3857700.0,32075400.0,4774060.0,NaN,1156340.0,34250.6,507483.0,79350.6,208598.0,61824.4,37354.5,186082.0,1510720.0,458306.0,17977.8,603685.0,210539.0,344149.0,9137820.0,1029290.0,108603.0,8667.28,578350.0,108005.0,9621.47,2607410.0,31886.0,100481.0,831886.0,12879.8,172731.0,54219.0,334859.0,1401860.0,29748.3,49802.5,1324370.0,23399.4,62755.1,22865.0,860978.0,125774.0,530914.0,1589560.0,26873.4,322370.0,9988.60,215991.0,430074.0,399489.0,140548.0,631794.0,11575700.0,84143.3,106498.0,104399.0,132270.0,50588.8,237956.0,33138.7,8755.34,572295.0,36107.4,44112.6,92071.9,91637.0,302429.0,47171.0,325214.0,36079.3,9451.71,438323.0,10886.2,1106490.0,112070.0,36232.9,528485.0,36406.9,27236.0,1673460.0,862077.0,922153.0,13998.0,89747.3,156923.0,NaN,56648.9,502305.0,2736190.0,12466.0,74564.3,31201.3,33921600.0,64855.6,27257.2,410727.0,15043.9,23457.2,380950.0,65910.6,43209.0,NaN,6323.72,23749.4,701219.0,449831.0,32505.8,21970.1,25613.8,39504.7,268657.0,1611130.0,39418.2,50763.0,663339.0,22053.5,NaN,3170450.0,34672.4,266247.0,9649.82,512157.0,161173.0,46305.4,1451950.0,221526.0,31592.7,18041.0,59467.7,31029.1,116200.0,519708.0,57246.2,10802.8,29027.7,36202.0,982965.0,200896.0,1282260.0,13189.3,633503.0,123757.0,34868.9,3417.71,34550.6,NaN,NaN,64488.0,22021.6,35217.8,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,NaN,29758.8,23833.7,18953.5,0,6.0,8.0,10.0,34.0,NaN,12.

# Modeling

In [14]:
# calculate month trend
def calculate_month_trend_predicitons(pred_month, trend, target):
    if target == 'updrs_4': 
        pred_month = pred_month.clip(54, None)
#         return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2])
        return trend[0] + pred_month * trend[1]
    elif target == "updrs_1":
        pred_month = pred_month.clip(2, None) # 42
#         return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2])
        return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2] + np.power(pred_month, 3) * trend[3])
#         return np.round((trend[0]) + pred_month * (trend[1]))
    elif target == 'updrs_2':
        pred_month = pred_month.clip(2, None)
#         return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2])
        return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2] + np.power(pred_month, 3) * trend[3])
#         return np.round((trend[0]) + pred_month * (trend[1]))
    elif target == "updrs_3":
        pred_month = pred_month.clip(2, None) # 83
#         return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2])
        return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2] + np.power(pred_month, 3) * trend[3])
#         return np.round((trend[0]) + pred_month * (trend[1]))
    


# best cv best lb
# target_to_trend = {'updrs_1': [5.266709591202201,
#   0.017276079706630247,
#   0.00010000004949866936,
#   9.539377244353253e-07],
#  'updrs_2': [6.1219755367144275,
#   0.004425008757718898,
#   0.00010000003654029528,
#   1.5274839889026747e-06],
#  'updrs_3': [21.173064314467048,
#   0.02342713599663579,
#   0.0007774623035543819,
#   7.146636076060611e-06],
# #  'updrs_4': [-2.1467602016903715, 0.043475487092279844]}
# 'updrs_4' : [-4.434453480103724, 0.07531448585334258]}


# test cv
target_to_trend = {'updrs_1': [5.84822783537646,
  0.016230229617191776,
  7.782056295354001e-05,
  -3.980507093123552e-07],
 'updrs_2': [6.202401128042322,
  -0.004133968297842226,
  9.404090722278564e-05,
  5.466368731943817e-06],
 'updrs_3': [21.360827985327084,
  0.03263037279565412,
  0.00031414682661494263,
  9.72316694865272e-06],
#  'updrs_4': [-0.72160402546765, 0.018037426033479218]}
 'updrs_4' : [-4.434453480103724, 0.07531448585334258]}
# 'updrs_4': [-1.695711389614207, 0.03356885153189329]}


# test cv, visit_count <=2 select, median max fillna, visit_month<=83 select
# target_to_trend = {'updrs_1': [6.255817432490031,
#   0.004194218661400878,
#   0.0001000000285842176,
#   5.016865966456184e-07],
#  'updrs_2': [6.206343175374049,
#   0.0043958628079618575,
#   9.418433172057456e-05,
#   1.440360648502827e-06],
#  'updrs_3': [21.4335295872646,
#   0.028895994583535835,
#   9.221791761484044e-05,
#   1.5014777135181147e-05],
#  'updrs_4': [-1.7765947297066382e-11, 0.00405314007562982]}
# 'updrs_4' : [-4.434453480103724, 0.07531448585334258]}


# target_to_trend = {'updrs_1': [5.80912042348557,
#   0.0173071854709929,
#   6.0964492262897784e-05,
#   -9.56428672689942e-08],
#  'updrs_2': [6.176952252326619,
#   0.0032738117683676596,
#   0.00019445529749482688,
#   1.754353083290596e-06],
#  'updrs_3': [21.3683140970437,
#   0.03893797302187778,
#   0.0007315169537536023,
#   6.003943094405298e-06],
#  'updrs_4': [-4.434453480103724, 0.07531448585334258]}


# target_to_trend = {
#      'updrs_1': [5.010363084341291, 0.039501096032359094],
#      'updrs_2': [5.359852200747447, 0.031143420111663673],
#      'updrs_3': [19.895914528688742, 0.1228511796843325],
#      'updrs_4': [-4.434453480103724, 0.07531448585334258]
#     }

    
    
# target_to_trend = {
# 'updrs_1': [5.085658338606598, 0.03412475104955939],
#  'updrs_2': [5.359852200747447, 0.031143420111663673],
#  'updrs_3': [21.38532421935566, 0.08563255211442955],
#  'updrs_4': [-4.434453480103724, 0.07531448585334258]
# }

# target_to_trend = {
#  'updrs_1': [5.085658338606598, 0.03412475104955939],
#  'updrs_2': [5.359852200747447, 0.031143420111663673],
#  'updrs_3': [21.393846648609884, 0.035271216724277576, 0.0011405504150867759],
#  'updrs_4': [-3.0448500382632755, 0.0358176811686638, 0.0003639474947207565]
# }

# target_to_trend = {
#  'updrs_1': [5.085658338606598, 0.03412475104955939],
#  'updrs_2': [5.359852200747447, 0.031143420111663673],
#  'updrs_3': [21.393846648609884, 0.035271216724277576, 0.0011405504150867759],
#  'updrs_4': [-4.434453480103724, 0.07531448585334258]
# }

# target_to_trend = {
#  'updrs_1': [5.085658338606598, 0.03412475104955939],
#  'updrs_2': [5.359852200747447, 0.031143420111663673],
#  'updrs_3': [21.403643055394227, 0.017703954745375883, 0.001845584215696705],
#  'updrs_4': [-4.541084710545025, 0.07488911424131216, 0.00012855347505502163]
# }



In [15]:
def calculate_predicitons_protein(protein, pred_month, protein_shift):
    trend_pred_month = target_to_trend[target]
    pred_month_trend = calculate_month_trend_predicitons(pred_month=pred_month, trend=trend_pred_month, target=target)
    return np.round(pred_month_trend + protein_shift)

def function_to_minimize(x):
    metric = smape_plus_1(
        y_true=y_true_array, 
        y_pred=calculate_predicitons_protein(
            protein=protein_array,
            pred_month=pred_month_array,
            protein_shift=x[0],
        )
    )
    return metric

In [16]:
def find_best_const(train_clinical_all_filtered, target):
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    global y_true_array
    global pred_month_array
    global protein_array
    y_true_array = train_clinical_all_filtered[columns_with_target].values.ravel()
    pred_month_array = train_clinical_all_filtered[columns_with_pred_month].values.ravel()
    protein_array = np.concatenate([train_clinical_all_filtered[feature].values] * 4)
#     protein_array = train_clinical_all_filtered[columns_with_feature_target].values.ravel()
    result = minimize(
        fun=function_to_minimize,
        x0=[0.0],
        method='Powell'
    ).x[0]
    
    del y_true_array, pred_month_array, protein_array
    return result

## Plot shifts

In [17]:
# corr_tt.sort_values("corr_sum")

In [18]:
# feature = 'P05060'
# feature = 'LGPLVEQGRVR'
# feature = 'P04217'
# feature = 'P05067'
# feature = 'Q06481'
# feature = 'P05060_diff_GYPGVQAPEDLEWER' # updrs_23
# feature = 'P05060_diff_SSQGGSLPSEEK' # updrs_14
# feature = 'O00533'
# feature = 'P00738'
feature = 'P05060'
quantiles = [0.0, 0.05, 0.95, 1.0]
# quantiles = [0.0, 0.05]
# quantiles = np.arange(0.0,1.1,0.1)

df_plot = []
for quantile_low, quantile_high in tqdm(zip(quantiles[:-1], quantiles[1:])):
    item = {
        'quantile_low': quantile_low,
        'quantile_high': quantile_high,
        'quantile_middle': (quantile_low + quantile_high) / 2
    }
    quantile_low_value = train_clinical_all[feature].quantile(quantile_low)
    quantile_high_value = train_clinical_all[feature].quantile(quantile_high)
    item['quantile_low_value'] = quantile_low_value
    item['quantile_high_value'] = quantile_high_value
    
    if quantile_high == 1:
        quantile_high_value += 0.00001
        
    train_clinical_all_filtered = train_clinical_all[
        (train_clinical_all[feature] >= quantile_low_value)
        & (train_clinical_all[feature] < quantile_high_value)
    ]
    for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        item[f'{target}_shift'] = find_best_const(train_clinical_all_filtered, target)
    df_plot.append(item)
    
df_plot = pd.DataFrame(df_plot)
df_plot

0it [00:00, ?it/s]

,quantile_low,quantile_high,quantile_middle,quantile_low_value,quantile_high_value,updrs_1_shift,updrs_2_shift,updrs_3_shift,updrs_4_shift
0,0.00,0.05,0.025,8040.76,99607.37,5.977736e+00,5.003212e+00,5.853169e+00,1.776595e-11
1,0.05,0.95,0.500,99607.37,438657.00,1.776595e-11,1.776595e-11,1.776595e-11,1.776595e-11
2,0.95,1.00,0.975,438657.00,672896.00,-7.673307e-01,-3.398385e+00,-6.787899e+00,1.776595e-11


In [19]:
# P05060
for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    fig = px.line(
        df_plot,
        y=f'{target}_shift',
        x='quantile_middle',
        title=feature + ' ' + target
    )
    fig.show()

## Find shifts

In [20]:
# # O15240
# for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
#     fig = px.line(
#         df_plot,
#         y=f'{target}_shift',
#         x='quantile_middle',
#         title=feature + ' ' + target
#     )
#     fig.show()

In [21]:
# single
npx_groups = [
    {'quantile_low': 0.00, 'quantile_high': 0.05},
    {'quantile_low': 0.05, 'quantile_high': 0.95},
    {'quantile_low': 0.95, 'quantile_high': 1.0},
]

# npx_groups = [
#     {'quantile_low': 0.00, 'quantile_high': 0.05},
#     {'quantile_low': 0.05, 'quantile_high': 0.25},
#     {'quantile_low': 0.25, 'quantile_high': 0.75},
#     {'quantile_low': 0.75, 'quantile_high': 0.95},
#     {'quantile_low': 0.95, 'quantile_high': 1.00},
# ]

# npx_groups = [
#     {'quantile_low': 0.00, 'quantile_high': 0.05},
#     {'quantile_low': 0.05, 'quantile_high': 0.50},
#     {'quantile_low': 0.50, 'quantile_high': 0.95},
#     {'quantile_low': 0.95, 'quantile_high': 1.00},
# ]

# npx_groups = [
#     {'quantile_low': 0.00, 'quantile_high': 0.10},
#     {'quantile_low': 0.10, 'quantile_high': 0.20},
#     {'quantile_low': 0.20, 'quantile_high': 0.30},
#     {'quantile_low': 0.30, 'quantile_high': 0.40},
#     {'quantile_low': 0.40, 'quantile_high': 0.50},
#     {'quantile_low': 0.50, 'quantile_high': 0.60},
#     {'quantile_low': 0.60, 'quantile_high': 0.70},
#     {'quantile_low': 0.70, 'quantile_high': 0.80},
#     {'quantile_low': 0.80, 'quantile_high': 0.90},
#     {'quantile_low': 0.90, 'quantile_high': 1.0},
# ]


# feature = 'P04180'
target_to_npx_groups_shift = defaultdict(list)

for target in ['updrs_1','updrs_2','updrs_3']:
    for npx_group in npx_groups:
        item = npx_group.copy()
        item['feature'] = feature
        
        if item['quantile_low'] == 0:
            item['quantile_low_value'] = -np.inf
        else:
            item['quantile_low_value'] = train_clinical_all[feature].quantile(item['quantile_low'])
            
        if item['quantile_high'] == 1:
            item['quantile_high_value'] = np.inf
        else: 
            item['quantile_high_value'] = train_clinical_all[feature].quantile(item['quantile_high'])

        train_clinical_all_filtered = train_clinical_all[
            (train_clinical_all[feature] >= item['quantile_low_value'])
            & (train_clinical_all[feature] < item['quantile_high_value'])
        ]
        
        item['shift'] = find_best_const(train_clinical_all_filtered, target)
        target_to_npx_groups_shift[target].append(item)

In [22]:
# # updrs_1
# npx_groups = [
#     {'quantile_low': 0.00, 'quantile_high': 0.04},
#     {'quantile_low': 0.04, 'quantile_high': 0.96},
#     {'quantile_low': 0.96, 'quantile_high': 1.0},
# ]

# updrs14_feature = 'P05060_diff_SSQGGSLPSEEK'
# updrs14_to_npx_groups_shift = defaultdict(list)

# for target in ['updrs_1', 'updrs_4']:
#     for npx_group in npx_groups:
#         item14 = npx_group.copy()
#         item14['feature'] = updrs14_feature
        
#         if item14['quantile_low'] == 0:
#             item14['quantile_low_value'] = -np.inf
#         else:
#             item14['quantile_low_value'] = train_clinical_all[updrs14_feature].quantile(item14['quantile_low'])
            
#         if item14['quantile_high'] == 1:
#             item14['quantile_high_value'] = np.inf
#         else: 
#             item14['quantile_high_value'] = train_clinical_all[updrs14_feature].quantile(item14['quantile_high'])

#         train_clinical_all_filtered = train_clinical_all[
#             (train_clinical_all[updrs14_feature] >= item14['quantile_low_value'])
#             & (train_clinical_all[updrs14_feature] < item14['quantile_high_value'])
#         ]
        
#         item14['shift'] = find_best_const(train_clinical_all_filtered, target)
#         updrs14_to_npx_groups_shift[target].append(item14)

In [23]:
# # updrs_2, updrs_3
# npx_groups = [
#     {'quantile_low': 0.0, 'quantile_high': 0.04},
#     {'quantile_low': 0.04, 'quantile_high': 0.96},
#     {'quantile_low': 0.96, 'quantile_high': 1.0},
# ]
# updrs23_feature = 'P05060_diff_GYPGVQAPEDLEWER'
# updrs23_to_npx_groups_shift = defaultdict(list)

# for target in ['updrs_2','updrs_3']:
#     for npx_group in npx_groups:
#         item23 = npx_group.copy()
#         item23['feature'] = updrs23_feature
        
#         if item23['quantile_low'] == 0:
#             item23['quantile_low_value'] = -np.inf
#         else:
#             item23['quantile_low_value'] = train_clinical_all[updrs23_feature].quantile(item23['quantile_low'])
            
#         if item23['quantile_high'] == 1:
#             item23['quantile_high_value'] = np.inf
#         else: 
#             item23['quantile_high_value'] = train_clinical_all[updrs23_feature].quantile(item23['quantile_high'])

#         train_clinical_all_filtered = train_clinical_all[
#             (train_clinical_all[updrs23_feature] >= item23['quantile_low_value'])
#             & (train_clinical_all[updrs23_feature] < item23['quantile_high_value'])
#         ]
        
#         item23['shift'] = find_best_const(train_clinical_all_filtered, target)
#         updrs23_to_npx_groups_shift[target].append(item23)

In [24]:
# target_to_npx_groups_shift

# Validation


In [25]:
train_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
supplemental_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

clinical_all = pd.concat([train_clinical, supplemental_clinical],axis=0).reset_index(drop=True)

# clinical_all[target_col] = clinical_all[target_col].apply(np.log1p)
proteins_features = pd.pivot_table(proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum').reset_index()


clinical_all = pd.merge(
    clinical_all,
    proteins_features,
    on= 'visit_id',
    how='left'
)

clinical_all[proteins_features.columns] = clinical_all.groupby('patient_id')[proteins_features.columns].\
                                                                                        fillna(method='ffill')

clinical_all['pred_month'] = clinical_all['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = clinical_all[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
    clinical_all = clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
clinical_all.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)




In [26]:
target_columns = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

sum_score = []
for target_column in target_columns:
    scores = []
    for plus_month in [0,6,12,24]:
        target = f"{target_column}"
        target_idx = clinical_all[f"{target}_plus_{plus_month}"].notna()
        df_train = clinical_all.loc[target_idx]
        pred_months = df_train['visit_month'] + plus_month
#         protein_array = df_train[f'{target}_plus_{plus_month}']
#         print(f'Target: {target_column} Dataset Shape: {df_train.shape}')


        linear_rate = calculate_month_trend_predicitons(
                pred_month=pred_months,
                trend=target_to_trend[target],
                target=target
            )

        for item in target_to_npx_groups_shift[target]:
            feature = item['feature']
            mask_index = df_train[
                (df_train[feature]>=item['quantile_low_value']) & \
                (df_train[feature]<item['quantile_high_value'])].index
            linear_rate[mask_index] += item['shift']

        linear_rate[linear_rate<=0]=0
    
        scores.append(smape_plus_1(df_train[f"{target}_plus_{plus_month}"], linear_rate))
    mean_score = np.mean(scores)
    sum_score.append(mean_score)
    print(f'Validation SMAPE: {mean_score:.4f}')
print(f"Sum Validation SMAPE: {np.mean(sum_score):.4f}")

Validation SMAPE: 54.4753
Validation SMAPE: 63.9118
Validation SMAPE: 57.5931
Validation SMAPE: 42.5400
Sum Validation SMAPE: 54.6301


In [27]:
target_columns = ['updrs_1', 'updrs_2', 'updrs_3','updrs_4']

sum_score = []
# print(f'feature 14 : {updrs14_feature}, feature 23 : {updrs23_feature}')
print(f'feature : {feature}')
for target_column in target_columns:
    plot_df = pd.DataFrame()
    fold_mean_score = []
    scores = []
    preds = []
    trues = []
    target = f"{target_column}"
    gkf = GroupKFold(n_splits=5)
#     gkf = GroupShuffleSplit(n_splits=5, random_state=1)
    groups = clinical_all['patient_id'].values
    
    for i, (trn_idx, val_idx) in enumerate(gkf.split(clinical_all, groups=groups)):
#         if target_column in ['updrs_1',"updrs_2","updrs_3"]:
        for plus_month in [0,6,12,24]:
            target_idx = clinical_all[f"{target}_plus_{plus_month}"].notna()
            df_train = clinical_all.iloc[val_idx].loc[target_idx]
            pred_months = df_train['visit_month'] + plus_month
    #             print(f'Target: {target_column} Dataset Shape: {df_train.shape}')

            linear_rate = calculate_month_trend_predicitons(
                    pred_month=pred_months,
                    trend=target_to_trend[target],
                    target=target
                )


            for item in target_to_npx_groups_shift[target]:
                feature = item['feature']
                mask_index = df_train[
                    (df_train[feature]>=item['quantile_low_value']) & \
                    (df_train[feature]<item['quantile_high_value'])].index
                linear_rate[mask_index] += item['shift']
#             if target == 'updrs_4':
#                 idx = pred_months==108
#                 display(linear_rate[idx])
#                 linear_rate[idx] -= 1
#                 display(linear_rate[idx])
#                 display(df_train[f"{target}_plus_{plus_month}"][idx])
            linear_rate[linear_rate<=0]=0
            
        
            preds.extend(linear_rate)
            trues.extend(df_train[f'{f"{target}_plus_{plus_month}"}'].values)
            scores.append(smape_plus_1(df_train[f"{target}_plus_{plus_month}"], linear_rate))
                

        fold_score = np.mean(scores)
        fold_mean_score.append(fold_score)
        print(f"Validation fold {i} score : {fold_score}")
    sum_score.append(np.mean(fold_mean_score))
    print(f'Validation {target} SMAPE: {np.mean(fold_mean_score):.4f}')
print(f"Sum Validation SMAPE: {np.mean(sum_score):.4f}")

feature : P05060
Validation fold 0 score : 54.00394646165674
Validation fold 1 score : 54.11550972594682
Validation fold 2 score : 53.49747445140375
Validation fold 3 score : 54.80810122100454
Validation fold 4 score : 54.462780082370614
Validation updrs_1 SMAPE: 54.1776
Validation fold 0 score : 59.47371088380119
Validation fold 1 score : 62.63898400215609
Validation fold 2 score : 62.38980002367782
Validation fold 3 score : 63.01420656757247
Validation fold 4 score : 63.90225051091984
Validation updrs_2 SMAPE: 62.2838
Validation fold 0 score : 54.81218526902602
Validation fold 1 score : 59.26871959566294
Validation fold 2 score : 58.46433615293805
Validation fold 3 score : 57.80372551394398
Validation fold 4 score : 57.59211452495443
Validation updrs_3 SMAPE: 57.5882
Validation fold 0 score : 47.78062615667643
Validation fold 1 score : 37.95425960535693
Validation fold 2 score : 41.691221971245916
Validation fold 3 score : 41.07479114823639
Validation fold 4 score : 42.5217080213567


In [28]:
# feature : P05060
# Validation fold 0 score : 54.00394646165674
# Validation fold 1 score : 54.11550972594682
# Validation fold 2 score : 53.49747445140375
# Validation fold 3 score : 54.80810122100454
# Validation fold 4 score : 54.462780082370614
# Validation updrs_1 SMAPE: 54.1776
# Validation fold 0 score : 59.47371088380119
# Validation fold 1 score : 62.63898400215609
# Validation fold 2 score : 62.38980002367782
# Validation fold 3 score : 63.01420656757247
# Validation fold 4 score : 63.90225051091984
# Validation updrs_2 SMAPE: 62.2838
# Validation fold 0 score : 54.81218526902602
# Validation fold 1 score : 59.26871959566294
# Validation fold 2 score : 58.46433615293805
# Validation fold 3 score : 57.80372551394398
# Validation fold 4 score : 57.59211452495443
# Validation updrs_3 SMAPE: 57.5882
# Validation fold 0 score : 47.78062615667643
# Validation fold 1 score : 37.95425960535693
# Validation fold 2 score : 41.691221971245916
# Validation fold 3 score : 41.07479114823639
# Validation fold 4 score : 42.5217080213567
# Validation updrs_4 SMAPE: 42.2045
# Sum Validation SMAPE: 54.0635

In [29]:
# target_columns = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

# seed_total_score = []
# updrs_1_score = []
# updrs_2_score = []
# updrs_3_score = []
# updrs_4_score = []
# # print(f'feature 14 : {updrs14_feature}, feature 23 : {updrs23_feature}')
# print(f'feature : {feature}')
# for seed in tqdm(np.arange(0, 200, 1)):
#     print(f"seed : {seed}")
#     sum_score = []
#     for target_column in target_columns:
#         plot_df = pd.DataFrame()
#         fold_mean_score = []
#         scores = []
#         preds = []
#         trues = []
#         target = f"{target_column}"
#     #     gkf = GroupKFold(n_splits=5)
#         gkf = GroupShuffleSplit(n_splits=5, random_state=seed)
#         groups = clinical_all['patient_id'].values
#         for i, (trn_idx, val_idx) in enumerate(gkf.split(clinical_all, groups=groups)):
# #             if target_column in ['updrs_1',"updrs_2","updrs_3"]:
#             for plus_month in [0,6,12,24]:
#                 target_idx = clinical_all[f"{target}_plus_{plus_month}"].notna()
#                 df_train = clinical_all.iloc[val_idx].loc[target_idx]
#                 pred_months = df_train['visit_month'] + plus_month
#         #             print(f'Target: {target_column} Dataset Shape: {df_train.shape}')

#                 linear_rate = calculate_month_trend_predicitons(
#                         pred_month=pred_months,
#                         trend=target_to_trend[target],
#                         target=target
#                     )


#                 for item in target_to_npx_groups_shift[target]:
#                     feature = item['feature']
#                     mask_index = df_train[
#                         (df_train[feature]>=item['quantile_low_value']) & \
#                         (df_train[feature]<item['quantile_high_value'])].index
#                     linear_rate[mask_index] += item['shift']
#                 linear_rate[linear_rate<=0]=0
# #                 if target == 'updrs_4':
# #                     display(linear_rate)
#                 preds.extend(linear_rate)
#                 trues.extend(df_train[f'{f"{target}_plus_{plus_month}"}'].values)
#                 scores.append(smape_plus_1(df_train[f"{target}_plus_{plus_month}"], linear_rate))

           
#             fold_score = np.mean(scores)
#             fold_mean_score.append(fold_score)
# #             print(f"Validation fold {i} score : {fold_score}")
#         sum_score.append(np.mean(fold_mean_score))
# #         print(f'Validation {target} SMAPE: {np.mean(fold_mean_score):.4f}')
#     seed_total_score.append(sum_score)
#     updrs_1_score.append(sum_score[0])
#     updrs_2_score.append(sum_score[1])
#     updrs_3_score.append(sum_score[2])
#     updrs_4_score.append(sum_score[3])
#     print(f"Sum Validation SMAPE: {np.mean(sum_score):.4f}")
# # print(f"Sum seed Validation SMAPE: {np.mean(seed_total_score):.4f}")

# seed_total_score = [np.mean(scores) for scores in seed_total_score]

In [30]:
# np.mean(seed_total_score)

In [31]:
# score_df = pd.DataFrame()
# score_df['total_score'] = seed_total_score
# score_df['updrs_1_score'] = updrs_1_score
# score_df['updrs_2_score'] = updrs_2_score
# score_df['updrs_3_score'] = updrs_3_score
# score_df['updrs_4_score'] = updrs_4_score

# score_df.hist(figsize=(16,12), grid=True)

In [32]:
# score_df = pd.DataFrame()
# score_df['total_score'] = seed_total_score
# score_df['updrs_1_score'] = updrs_1_score
# score_df['updrs_2_score'] = updrs_2_score
# score_df['updrs_3_score'] = updrs_3_score
# score_df['updrs_4_score'] = updrs_4_score

# score_df.hist(figsize=(16,12), grid=True)

In [33]:
# score_df.describe()

## Predictions

In [34]:
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

proteins_features_all = pd.DataFrame()
peptides_features_all = pd.DataFrame()
# The API will deliver four dataframes in this specific order:
for test_clinical_data, test_peptides, test_proteins, sample_submission in iter_test:
    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
    sample_submission['visit_id'] = sample_submission['patient_id'].astype(str) + '_' + sample_submission['visit_month'].astype(str)

    # proteins
    proteins_features = pd.pivot_table(test_proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')
    proteins_features['visit_id'] = proteins_features.index
    proteins_features_all = pd.concat([proteins_features_all, proteins_features])
    proteins_features_all['patient_id'] = proteins_features_all.index.map(lambda x: int(x.split('_')[0]))
    proteins_features_all[proteins_features.columns] = proteins_features_all.groupby('patient_id')[proteins_features.columns].fillna(method='ffill')
    proteins_features = proteins_features_all.groupby('patient_id', as_index=False).last()
    
    
    
    sample_submission = sample_submission.merge(
        proteins_features,
        on='patient_id',
        how='left'
    )

    for i in range(1, 5):
        target = f'updrs_{i}'
        mask_target = sample_submission['target_name'] == target

        sample_submission.loc[mask_target, 'rating'] = calculate_month_trend_predicitons(
            pred_month=sample_submission.loc[mask_target, 'pred_month'],
            trend=target_to_trend[target],
            target=target
        )
        
        
        for item in target_to_npx_groups_shift[target]:
            feature = item['feature']
            mask_feature_range = mask_target & (
                (sample_submission[feature] >= item['quantile_low_value'])
                & (sample_submission[feature] < item['quantile_high_value'])
            )
            sample_submission.loc[mask_feature_range, 'rating'] += item['shift']
#             sample_submission.loc[mask_feature_range, 'rating'][sample_submission.loc[mask_feature_range, 'rating']<=0] = 0 
        sample_submission.loc[mask_target, 'rating'] = np.round(sample_submission.loc[mask_target, 'rating'])
            
#         else:
#             mask_target = sample_submission['target_name'] == target
#             pred_months = sample_submission.loc[mask_target, 'pred_month']

#             sample_submission.loc[mask_target, 'rating'] = calculate_month_trend_predicitons(
#                 pred_month=sample_submission.loc[mask_target, 'pred_month'],
#                 trend=target_to_trend[target],
#                 target=target
#             )

#             sample_submission.loc[mask_target, 'rating'] = np.round(sample_submission.loc[mask_target, 'rating'])
#             for item in updrs23_to_npx_groups_shift[target]:
#                 feature = item['feature']
#                 mask_feature_range = mask_target & (
#                     (sample_submission[feature] >= item['quantile_low_value'])
#                     & (sample_submission[feature] < item['quantile_high_value'])
#                 )
#                 sample_submission.loc[mask_feature_range, 'rating'] += item['shift']
#             sample_submission.loc[mask_target, 'rating'] = np.round(sample_submission.loc[mask_target, 'rating'])
        
#         if target == 'updrs_4':
#             display(sample_submission.loc[mask_target, 'rating'].values)
        
#         else:
#             mask_target = sample_submission['target_name'] == target
#             pred_months = sample_submission.loc[mask_target, 'pred_month']
            
#             zero_score = np.array([0]*len(pred_months))
            # median score
#             median_missing_idxs = np.array([np.argmin(np.abs( pred_month - target_visit_month_medians['visit_month'])) for pred_month in pred_months])
#             median_rate = np.array([target_visit_month_medians.iloc[idx][f'{target}'] for idx in median_missing_idxs])
                    
#             sample_submission.loc[mask_target, 'rating'] = median_rate
            
    
    # call the env.predict for every iteration
    env.predict(sample_submission[['prediction_id', 'rating']])
    display(sample_submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,prediction_id,rating,patient_id,visit_month,target_name,plus_month,pred_month,visit_id_x,O00391,O00533,O00584,O14773,O14791,O15031,O15240,O15394,O43505,O60888,O75144,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01593,P01594,P01608,P01621,P01717,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05452,P05546,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09382,P09486,P09871,P0C0L5,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17900,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P22692,P23083,P23142,P24592,P25311,P27169,P28074,P30086,P31997,P35542,P36222,P36955,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80723,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13228,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q969P0,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,visit_id_y
0,3342_0_updrs_1_plus_0_months,6.0,3342,0,updrs_1,0,0,3342_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3342_0_updrs_1_plus_6_months,6.0,3342,0,updrs_1,6,6,3342_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3342_0_updrs_1_plus_12_months,6.0,3342,0,updrs_1,12,12,3342_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

,prediction_id,rating,patient_id,visit_month,target_name,plus_month,pred_month,visit_id_x,O00391,O00533,O00584,O14773,O14791,O15031,O15240,O15394,O43505,O60888,O75144,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01593,P01594,P01608,P01621,P01717,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,P02751,P02753,P02760,P02763,P02765,P02766,P02768,P02774,P02787,P02790,P02792,P04004,P04075,P04156,P04180,P04196,P04207,P04211,P04216,P04217,P04275,P04406,P04433,P05060,P05067,P05090,P05155,P05156,P05452,P05546,P06396,P06454,P06681,P06727,P07195,P07225,P07333,P07339,P07602,P07711,P07858,P07998,P08123,P08133,P08253,P08294,P08493,P08571,P08603,P08637,P08697,P09104,P09382,P09486,P09871,P0C0L5,P10451,P10643,P10645,P10909,P11142,P11277,P12109,P13473,P13521,P13591,P13611,P13671,P13987,P14174,P14314,P14618,P16035,P16070,P16152,P16870,P17174,P17900,P17936,P18065,P19021,P19652,P19823,P19827,P20774,P20933,P22692,P23083,P23142,P24592,P25311,P27169,P28074,P30086,P31997,P35542,P36222,P36955,P39060,P40925,P41222,P43121,P43251,P43652,P49588,P49908,P51884,P54289,P55290,P60174,P61278,P61626,P61769,P61916,P80723,P80748,P98160,Q02818,Q06481,Q08380,Q12805,Q12841,Q12907,Q13228,Q13283,Q13332,Q13449,Q13451,Q13740,Q14118,Q14508,Q14515,Q14624,Q15904,Q16270,Q16610,Q562R1,Q6UX71,Q6UXB8,Q6UXD5,Q7Z3B1,Q7Z5P9,Q8IWV7,Q8N2S1,Q8NBJ4,Q8NE71,Q92520,Q92823,Q92876,Q969P0,Q96BZ4,Q96KN2,Q96PD5,Q96S96,Q99435,Q99674,Q99969,Q9BY67,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7,visit_id_y,O14498,O75326,P01780,P30530,P32754,P36980,P54819,Q03167,Q99683
0,3342_6_updrs_1_plus_0_months,6.0,3342,6,updrs_1,0,6,3342_6,14564.0,616804.0,61418.1,14959.2,2853.12,NaN,223373.0,58863.0,207497.0,175485.0,160410.0,69440.7,282667.0,1159120.0,652144.0,115974.0,5200440.0,107976.0,361287.0,61713.9,411366.0,2497440.0,16205500.0,1708870.0,2324710.0,1846170.0,3909780.0,11004.3,NaN,28752900.0,1010440.0,87698.2,14954.20,14129.4,NaN,9509.49,134818.0,111967.0,NaN,2886780.0,5598370.0,2032630.0,530162.0,15388.7,2498500.0,113733.0,47920.2,9133330.0,13761300.0,455862.0,19889.2,55558.1,778884.0,714271.0,242512.0,161767.0,81046.1,1170730.0,314772.0,2099250.0,442484.0,356842.0,4930670.0,2438980.0,19436900.0,295860000.0,3612060.0,37621900.0,5032040.0,17495.2,1082240.0,29788.1,455989.0,75907.3,218839.0,114213.0,15641.9,380529.0,1676090.0,459627.0,7750.06,685257.0,407709.0,981028.0,7433310.0,1119460.0,58624.6,372610.0,92338.1,2289100.0,21742.8,82169.6,616319.0,10108.00,130596.0,NaN,262013.0,1472470.0,52825.3,50683.8,1361510.0,15682.4,NaN,21935.3,909967.0,48546.6,476188.0,1528130.0,35636.4,247276.0,7757.99,6524.61,127996.0,391219.0,18111.7,490139.0,211077.0,752113.0,9056370.0,75469.6,NaN,79162.7,221192.0,83829.4,244919.0,28180.1,8065.04,503897.0,32845.8,20554.0,87254.1,50899.1,277042.0,160021.0,79710.3,36385.8,46232.2,7636.82,169464.0,14569.60,649979.0,155016.0,29268.9,378671.0,9306.34,55359.8,28592.4,1278060.0,853869.0,751672.0,35315.0,87206.2,80416.4,573000.0,76689.5,366985.0,2603100.0,58308.6,34123.1,59234100.0,43855.2,52303.4,300389.0,12130.5,78331.4,339022.0,64392.5,52286.6,7252.54,8931.03,9183.23,410546.0,309693.0,7994.16,43491.9,15152.8,24866.7,23946.0,289318.0,1387110.0,17864.8,49900.7,NaN,480429.0,24924.8,45426.9,2761080.0,31242.90,197321.0,7815.59,495632.0,163118.0,55870.8,692729.0,280747.0,13941.5,18345.0,42463.6,NaN,118481.0,474377.0,89461.1,6057.78,26572.0,NaN,1029960.0,240527.0,1364170.0,15135.4,12589.8,1190480.0,95103.2,34384.9,3713.61,31705.1,47999.1,64253.8,173205.0,46411.8,111979.0,16836.0,40179.3,325226.0,64411.5,25117.5,51473.3,14624.5,3342_6,27830.1,5967.92,18883.7,36424.2,86222.4,2886.27,121392.0,6911.05,98810.7
1,3342_6_updrs_1_plus_6_months,6.0,3342,6,updrs_1,6,12,3342_6,14564.0,616804.0,61418.1,14959.2,2853.12,NaN,223373.0,58863.0,207497.0,175485.0,160410.0,69440.7,28

In [35]:
target_to_npx_groups_shift

defaultdict(list,
            {'updrs_1': [{'quantile_low': 0.0,
               'quantile_high': 0.05,
               'feature': 'P05060',
               'quantile_low_value': -inf,
               'quantile_high_value': 99607.37000000001,
               'shift': 5.977735873499518},
              {'quantile_low': 0.05,
               'quantile_high': 0.95,
               'feature': 'P05060',
               'quantile_low_value': 99607.37000000001,
               'quantile_high_value': 438657.0,
               'shift': 1.776594685014621e-11},
              {'quantile_low': 0.95,
               'quantile_high': 1.0,
               'feature': 'P05060',
               'quantile_low_value': 438657.0,
               'quantile_high_value': inf,
               'shift': -0.7673307493416143}],
             'updrs_2': [{'quantile_low': 0.0,
               'quantile_high': 0.05,
               'feature': 'P05060',
               'quantile_low_value': -inf,
               'quantile_high_value': 99607

In [36]:
# mask_target = sample_submission['target_name'] == target
# pred_months = sample_submission.loc[mask_target, 'pred_month']
# ratings = []
# display(pred_months)
# for pred_month in pred_months:
#     if pred_month not in target_visit_month_medians['visit_month'].values:
#         median_missing_idx = np.argmin(np.abs(pred_month - target_visit_month_medians['visit_month']))
#         rating = target_visit_month_medians.iloc[median_missing_idx][f'{target}']
#         ratings.extend(rating)
#     else:
#         rating = target_visit_month_medians[target_visit_month_medians['visit_month']==pred_month][f"{target}"].values
#         ratings.extend(rating)
    
# display(np.round(ratings))
# df = pd.DataFrame(columns=['label'])
# df['label'] = np.array(ratings)
# display(df)
# sample_submission.loc[mask_target, 'rating'] = np.array(ratings)

In [37]:
# sample_submission